In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datasets import Dataset
import numpy as np

In [ ]:
data = pd.read_csv("/kaggle/input/tweet-dataset/Tweets Dataset.csv")
data.head()

In [ ]:
data['Party'] = data['Party'].map({'Democrat' : 0, 'Republican' : 1})

In [ ]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

In [ ]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["Tweet"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

In [ ]:
# Set the format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'Party'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'Party'])

In [ ]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [ ]:
# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

In [ ]:
# Set up the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * 3  # 3 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [ ]:
# Training loop
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
def load_model():
    model = BertForSequenceClassification.from_pretrained("/kaggle/input/bert-tweet")
    tokenizer = BertTokenizer.from_pretrained("/kaggle/input/token-tweet")
    model.to(device)
    model.eval()
    return model, tokenizer

In [ ]:
model, tokenizer = load_model()

In [ ]:
# Predict gender function (Male or Female)
def predict_male_or_female(comment, model, tokenizer, device):
    model.eval()
    encoding = tokenizer(comment, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        prediction = torch.argmax(outputs.logits, dim=1).cpu().item()
    return "Democrat" if prediction == 0 else "Republican"

In [ ]:
# Predict gender probability function (Male, Female probabilities)
def predict_gender_prob(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        softmax = torch.nn.Softmax(dim=1)
        probabilities = softmax(outputs.logits).cpu().numpy()
    return probabilities

In [ ]:
# Predict instance function (Wrapper to call predict_gender_prob)
def predict_instance(text):
    preds = predict_gender_prob(text, model, tokenizer, device)
    # Return as dictionary with 'Male' and 'Female' probabilities
    return preds

In [ ]:
# Example usage
txt = "I've had the 50watter since Oct of last year and I'm still impressed and blown away every time I play it. It's unreal. Killer choice"
predict_male_or_female(txt, model, tokenizer, device)

In [ ]:
txt = "You l've got a good man there hun. Take care of each other and it'll last a long time.\nFor sore throats my dad used to take 2 tablespoons of apple cider vinegar and the same amount of honey, mix it in at least 8oz of hot water. Drink it while ot's still hot, but not burning. Worked every time for me...still does."
predict_male_or_female(txt, model, tokenizer, device)

In [ ]:
from lime.lime_text import LimeTextExplainer, IndexedString
import numpy as np
from operator import itemgetter
from tqdm import tqdm
import csv

In [ ]:
# LIME explainer
class_names = ['Democrat', 'Republican']
explainer = LimeTextExplainer(class_names=class_names, random_state=42)

In [ ]:
# Sort tuples array by second item
def sort_tuples_array_by_second_item(tuples):
    return sorted(tuples, key=itemgetter(1))

In [ ]:
# Get max explained words function
def get_max_explained_words(txt, explainer_num_samples=100):
    prediction = predict_male_or_female(txt, model, tokenizer, device)
    prediction_label = 0 if prediction == "Democrat" else 1

    indexed_string = IndexedString(txt, bow=explainer.bow, split_expression=explainer.split_expression,
                                 mask_string=explainer.mask_string) # Access from explainer object
    if indexed_string.num_words() == 0:
        print(f"Warning: Skipping tweet with no recognized words: {txt}")
        return words, wordsForCSV

    exp = explainer.explain_instance(txt, predict_instance, num_samples=explainer_num_samples)
    exp_list = []
    for x in zip(exp.local_exp[1], exp.as_list()):
        exp_list.append((x[1][0], x[1][1], x[0][0]))

    # Features with negative score are for Male class
    male_list = list(filter(lambda x: x[1] < 0, exp_list))
    male_list = sort_tuples_array_by_second_item(male_list)

    # Features with positive score are for Female class
    female_list = list(filter(lambda x: x[1] > 0, exp_list))
    female_list = sort_tuples_array_by_second_item(female_list)

    # If comment predicted Male
    if prediction_label == 0:
        if len(male_list) > 1:
            male_mc = male_list[0]
            if (male_mc[0], 0) in words:
                words[(male_mc[0], 0)]['lime_score'].append(male_mc[1])
                words[(male_mc[0], 0)]['position'] = male_mc[2]
            else:
                words[(male_mc[0], 0)] = {}
                words[(male_mc[0], 0)]['lime_score'] = [male_mc[1]]
                words[(male_mc[0], 0)]['position'] = male_mc[2]
                wordsForCSV.append([male_mc[0], 0, male_mc[1]])

            male_mc = male_list[1]
            if (male_mc[0], 0) in words:
                words[(male_mc[0], 0)]['lime_score'].append(male_mc[1])
                words[(male_mc[0], 0)]['position'] = male_mc[2]
            else:
                words[(male_mc[0], 0)] = {}
                words[(male_mc[0], 0)]['lime_score'] = [male_mc[1]]
                words[(male_mc[0], 0)]['position'] = male_mc[2]
                wordsForCSV.append([male_mc[0], 0, male_mc[1]])
    else:
        if len(female_list) > 1:
            female_mc = female_list[(len(female_list)-1)]
            if (female_mc[0], 1) in words:
                words[(female_mc[0], 1)]['lime_score'].append(female_mc[1])
                words[(female_mc[0], 1)]['position'] = female_mc[2]
            else:
                words[(female_mc[0], 1)] = {}
                words[(female_mc[0], 1)]['lime_score'] = [female_mc[1]]
                words[(female_mc[0], 1)]['position'] = female_mc[2]
                wordsForCSV.append([female_mc[0], 1, female_mc[1]])

            female_mc = female_list[(len(female_list)-2)]
            if (female_mc[0], 1) in words:
                words[(female_mc[0], 1)]['lime_score'].append(female_mc[1])
                words[(female_mc[0], 1)]['position'] = female_mc[2]
            else:
                words[(female_mc[0], 1)] = {}
                words[(female_mc[0], 1)]['lime_score'] = [female_mc[1]]
                words[(female_mc[0], 1)]['position'] = female_mc[2]
                wordsForCSV.append([female_mc[0], 1, female_mc[1]])

    return words, wordsForCSV

In [ ]:
# Load data from CSV
def load_original_data(data_file):
    df = pd.read_csv(data_file)
    df.fillna("", inplace=True)  # Replace NaN values with empty strings
    comments = df['Tweet'].tolist()
    genders = df['Party'].map(lambda x: 0 if x == "Democrat" else 1).tolist()
    return comments, genders

In [ ]:
# Load original data
original_comments, original_genders = load_original_data('/kaggle/input/tweet-dataset/Tweets Dataset.csv')

In [ ]:
n = int(len(original_comments) / 3)
x = [original_comments[i:i + n] for i in range(0, len(original_comments), n)]
y = [original_genders[i:i + n] for i in range(0, len(original_genders), n)]

In [ ]:
# Initialize variables
header = ["word", "label", "limescore"]
file_path = '/kaggle/working/file.csv'
words = {}
wordsForCSV = []

In [ ]:
for comment in tqdm(x[0], total=len(x[0])):
    words, wordsForCSV = get_max_explained_words(comment)

In [ ]:
# Write results to CSV
with open(file_path, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(wordsForCSV)

In [ ]:
for comment in tqdm(x[1], total=len(x[1])):
    words, wordsForCSV = get_max_explained_words(comment)

with open(file_path, 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(wordsForCSV)

In [ ]:
for comment in tqdm(x[2], total=len(x[2])):
    words, wordsForCSV = get_max_explained_words(comment)

with open(file_path, 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(wordsForCSV)